In [31]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
from splinter import Browser
import os

In [9]:
# URL of page to be scraped
#url = 'https://www.walgreens.com/store/c/halloween-candy/ID=520931-tier3'
url = 'https://www.walgreens.com/store/store/category/productlist.jsp?N=520931&Eon=520931'


In [32]:
# Read HTML from file
filepath = os.path.join("Resources", "walgreens.html")
with open(filepath) as file:
    wal_html = file.read()

In [33]:
#browser.visit(url)
html = browser.html
soup = BeautifulSoup(wal_html, 'html.parser')


In [34]:
# <div class="brand" id="prodBrandNamecompare_sku6253945" data-reactid="468">PayDay</div>
# "brand"
#<input type="checkbox" id="filter_353669" tabindex="0" value="353669" name="353669" title="Sour Punch(2)">
brands = soup.find_all("div", class_="item card card__product")
print(brands)

t store</span></div><div class="avail-cta" id="not-available-for-shippingcompare_sku6151899" name="not-available-for-shipping compare_sku6151899"><!-- react-text: 4398 -->Not available for shipping<!-- /react-text --><span class="sr-only">Not available for shipping</span></div><button class="btn btn__blue" id="pickup-ship-btncompare_sku6151899" name="pickup-ship-btn"><!-- react-text: 4401 -->Pick up<!-- /react-text --><span class="sr-only">will open overlay</span></button></div></div><span aria-live="polite" class="sr-only" id="ownbrandAnnounce"></span></div>, <div class="item card card__product" id="productcard300400854"><div aria-hidden="true" id="productcardheight300400854" style="text-align: center;"><a href="/store/c/haribo-sweet-%26-scary-mix-assorted,-laydown-bag/ID=300400854-product" id="productOmniSelectcompare_400622247" tabindex="-1"><figure class="product__img"></figure><span class="sr-only"><!-- react-text: 4410 -->Haribo Sweet &amp; Scary Mix Assorted, Laydown Bag<!-- /re

In [35]:
len(brands)

72

In [38]:
brands_list = []
for brand in brands:
    
    brands_list.append(brand.text)
    print(brand.text)

PayDay
Snickers
Tootsie Roll
Milky Way
Reese's
Mounds
Hershey's
Kisses


In [57]:
#<a name="filter_by_Brand" class="no-toggle title__master dt-selected" id="filter_Brand" href="javascript:void(0)" aria-expanded="true" data-reactid="173"><strong data-reactid="174"><legend data-reactid="175">Brand</legend></strong></a>
# type something that allows you select the a element (link that has the id)
# for the a element, find the next sibling
# tier list of things, nested inside things, a tree
# a , div, div, ul, li, 

In [58]:
labels = soup.find_all('li', 'check-box')
print(labels)

for label in labels:
    print(label.text)

[<li class="check-box" data-reactid="112"><input data-reactid="113" id="filter_2000011813" name="2000011813" tabindex="0" title="Candy(78)" type="checkbox" value="2000011813"/><label data-reactid="114" for="filter_2000011813"><span class="check" data-reactid="115"></span><span class="text-truncate" data-reactid="116">Candy</span><!-- react-text: 117 --> (<!-- /react-text --><!-- react-text: 118 -->78<!-- /react-text --><!-- react-text: 119 -->)<!-- /react-text --></label></li>, <li class="check-box" data-reactid="120"><input data-reactid="121" id="filter_2000013458" name="2000013458" tabindex="0" title="Chocolate - Bag(49)" type="checkbox" value="2000013458"/><label data-reactid="122" for="filter_2000013458"><span class="check" data-reactid="123"></span><span class="text-truncate" data-reactid="124">Chocolate - Bag</span><!-- react-text: 125 --> (<!-- /react-text --><!-- react-text: 126 -->49<!-- /react-text --><!-- react-text: 127 -->)<!-- /react-text --></label></li>, <li class="chec

In [59]:
what = soup.label.parent
print(what.text)

Search by keyword or item #Closing Suggestion 


In [60]:
filterA = soup.find('a',id='filter_Brand')
print(filterA)

<a aria-expanded="true" class="no-toggle title__master dt-selected selected" data-reactid="173" href="javascript:void(0)" id="filter_Brand" name="filter_by_Brand"><strong data-reactid="174"><legend data-reactid="175">Brand</legend></strong></a>


In [61]:
parentDiv = filterA.find_next_sibling('div')
labeleds = parentDiv.find_all(class_="text-truncate")
for labeled in labeleds:
    print(labeled.text)

Hershey's
Mars
Reese's
Brach's
Russell Stover
Kit Kat
M&M's


In [62]:
#splinter is what's used to navigate a page


In [30]:
ratings = soup.find_all("span", class_="product__rating")
print(ratings)

[<span class="product__rating" data-reactid="488"><figure data-reactid="489"><img alt="4.6 out of 5 total 12 reviews" data-reactid="490" id="reviewbazarcompare_sku6253945" src="//www.walgreens.com/images/adaptive/bvrating/rating-4_6.gif" title="4.6 out of 5"><figcaption data-reactid="491">12</figcaption></img></figure></span>, <span class="product__rating" data-reactid="542"><figure data-reactid="543"><img alt="5.0 out of 5 total 1 reviews" data-reactid="544" id="reviewbazarcompare_sku6062317" src="//www.walgreens.com/images/adaptive/bvrating/rating-5_0.gif" title="5.0 out of 5"><figcaption data-reactid="545">1</figcaption></img></figure></span>, <span class="product__rating" data-reactid="591"><figure data-reactid="592"><img alt="5.0 out of 5 total 6 reviews" data-reactid="593" id="reviewbazarcompare_sku6180836" src="//www.walgreens.com/images/adaptive/bvrating/rating-5_0.gif" title="5.0 out of 5"><figcaption data-reactid="594">6</figcaption></img></figure></span>, <span class="produc

In [39]:
ratings_list = []
for rating in ratings:
    ratings_list.append(rating.img["alt"])
    print(rating.img["alt"])

4.6 out of 5 total 12 reviews
5.0 out of 5 total 1 reviews
5.0 out of 5 total 6 reviews
4.9 out of 5 total 43 reviews
5.0 out of 5 total 3 reviews
4.8 out of 5 total 16 reviews
3.9 out of 5 total 36 reviews
5.0 out of 5 total 1 reviews


In [42]:
df = pd.DataFrame({"brands": brands_list, "ratings": ratings_list})
df

,brands,ratings
0,PayDay,4.6 out of 5 total 12 reviews
1,Snickers,5.0 out of 5 total 1 reviews
2,Tootsie Roll,5.0 out of 5 total 6 reviews
3,Milky Way,4.9 out of 5 total 43 reviews
4,Reese's,5.0 out of 5 total 3 reviews
5,Mounds,4.8 out of 5 total 16 reviews
6,Hershey's,3.9 out of 5 total 36 reviews
7,Kisses,5.0 out of 5 total 1 reviews


In [79]:
individual_ratings = []
reviews = []
for string in df['ratings']:
    # extract the individual rating
    rating = string[0:3]
    individual_ratings.append(float(rating))
    # extract number of reviews    
    count = [int(i) for i in string.split() if i.isnumeric()]
    reviews.append(count[1])
reviews

[12, 1, 6, 43, 3, 16, 36, 1]

In [80]:
individual_ratings

[4.6, 5.0, 5.0, 4.9, 5.0, 4.8, 3.9, 5.0]

In [82]:
df['individual_ratings'] = individual_ratings
df['reviews'] = reviews
df.head()

,brands,ratings,individual_ratings,reviews
0,PayDay,4.6 out of 5 total 12 reviews,4.6,12
1,Snickers,5.0 out of 5 total 1 reviews,5.0,1
2,Tootsie Roll,5.0 out of 5 total 6 reviews,5.0,6
3,Milky Way,4.9 out of 5 total 43 reviews,4.9,43
4,Reese's,5.0 out of 5 total 3 reviews,5.0,3


https://towardsdatascience.com/elevate-your-webscraping-with-splinter-a926eee7f7d9